In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, default_collate
from sklearn.preprocessing import LabelEncoder

## Read Data and convert them to numpy arrays using .values

In [9]:
data = pd.read_csv('sonar_csv.csv')
X = data.iloc[1:, 0:60].values
y = data.iloc[1:, 60].values
X.shape, y.shape

((207, 60), (207,))

## Encode class values as integers

In [12]:
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y).reshape(-1, 1)

In [13]:
class SonarDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        features = self.X[idx]
        target = self.y[idx]
        return features, target

In [17]:
dataset = SonarDataset(X, y)
trainset, testset = random_split(dataset, [0.7, 0.3])
loader = DataLoader(trainset, batch_size=16, shuffle=True)

In [15]:
model = nn.Sequential(
    nn.Linear(60, 60),
    nn.ReLU(),
    nn.Linear(60, 30),
    nn.ReLU(),
    nn.Linear(30, 1),
    nn.Sigmoid()
)

In [22]:
n_epochs = 200
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()
for epoch in range(n_epochs):
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        y_pred = model(X_batch.float())
        loss = loss_fn(y_pred, y_batch.float())
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}, Accuracy: {((y_pred > 0.5) == y_batch).sum().item() / len(y_batch)}')

Epoch: 0, Loss: 0.003474641125649214, Accuracy: 1.0
Epoch: 10, Loss: 0.09941381961107254, Accuracy: 1.0
Epoch: 20, Loss: 0.02185257151722908, Accuracy: 1.0
Epoch: 30, Loss: 0.02030925266444683, Accuracy: 1.0
Epoch: 40, Loss: 0.004565887618809938, Accuracy: 1.0
Epoch: 50, Loss: 0.0026962007395923138, Accuracy: 1.0
Epoch: 60, Loss: 6.182855577208102e-05, Accuracy: 1.0
Epoch: 70, Loss: 1.8494847608963028e-06, Accuracy: 1.0
Epoch: 80, Loss: 0.00877311360090971, Accuracy: 1.0
Epoch: 90, Loss: 5.650610546581447e-05, Accuracy: 1.0
Epoch: 100, Loss: 0.000318039528792724, Accuracy: 1.0
Epoch: 110, Loss: 3.612783257267438e-05, Accuracy: 1.0
Epoch: 120, Loss: 1.6808651707833633e-05, Accuracy: 1.0
Epoch: 130, Loss: 0.01765795238316059, Accuracy: 1.0
Epoch: 140, Loss: 0.0, Accuracy: 1.0
Epoch: 150, Loss: 0.0018390753539279103, Accuracy: 1.0
Epoch: 160, Loss: 4.0742342832800205e-10, Accuracy: 1.0
Epoch: 170, Loss: 0.0, Accuracy: 1.0
Epoch: 180, Loss: 1.0081317469712303e-07, Accuracy: 1.0
Epoch: 190,

In [23]:
X_test, y_test = default_collate(testset)
model.eval()
y_pred = model(X_test.float())
acc = ((y_pred > 0.5) == y_test).sum().item() / len(y_test)
print(f'Accuracy: {acc}')

Accuracy: 0.7903225806451613
